In [ ]:
pip install --upgrade pyarrow google-cloud-bigquery

In [ ]:
%pip install -q google-cloud-bigquery pandas-gbq requests matplotlib

In [ ]:
import requests
import pandas as pd
from pandas_gbq import to_gbq
from google.colab import auth
from google.cloud import bigquery
import numpy as np
import time
from datetime import datetime

In [ ]:
# Define the parameters (same as provided)
regions = ['de', 'us-ny', 'gb']
payment_methods = {
    'de': ['khipu', 'sepa-bank-transfer', 'debit-credit-card'],
    'gb': ['gbp-bank-transfer', 'debit-credit-card', 'sepa-bank-transfer'],
    'us-ny': ['instant-bank-transfer', 'debit-credit-card']
}
crypto_ids = {
    'Ethereum Mainnet': '/currencies/crypto/1/0x0000000000000000000000000000000000000000',
    'USDT (Ethereum)': '/currencies/crypto/1/0xdac17f958d2ee523a2206206994597c13d831ec7',
    'USDT (BNB Chain)': '/currencies/crypto/56/0x55d398326f99059ff775485246999027b3197955'
}
fiat_currencies = {'de': '/currencies/fiat/eur', 'us-ny': '/currencies/fiat/usd', 'gb': '/currencies/fiat/gbp'}

# Define crypto name mapping (same as provided)
crypto_name_mapping = {
    '/currencies/crypto/1/0x0000000000000000000000000000000000000000': 'Ethereum Mainnet',
    '/currencies/crypto/1/0xdac17f958d2ee523a2206206994597c13d831ec7': 'USDT (Ethereum)',
    '/currencies/crypto/56/0x55d398326f99059ff775485246999027b3197955': 'USDT (BNB Chain)'
}

# Hybrid approach for generating sample points (same as provided)
low_range = np.arange(30, 101, 10)
mid_range = np.arange(200, 1001, 200)
high_range = np.arange(2000, 30001, 3000)

# Combine all ranges
price_points = np.concatenate((low_range, mid_range, high_range))
# price_points = [30,100]  # Use a single price point for testing

def fetch_data(region, payment_method, crypto_id, fiat_id, amount):
    common_providers = [
        '/providers/moonpay-b',
        '/providers/banxa',
        '/providers/unlimitmeld',
        '/providers/transak-b',
        '/providers/mercuryo',
        '/providers/ramp-network',
        '/providers/transfimeld',
        '/providers/revolut',
        '/providers/sardine-b',
        '/providers/blockchaindotcom',
        '/providers/stripe',
        '/providers/coinbase',
        '/providers/onramp-money',
    ]
    
    if payment_method == 'khipu':
        providers = common_providers + ['/providers/onramp-money']
    else:
        providers = common_providers
    
    providers_str = '&'.join([f'providers%5B{i}%5D={provider}' for i, provider in enumerate(providers)])
    url = (f"https://on-ramp.api.cx.metamask.io/providers/all/quote?"
           f"regionId=%2Fregions%2F{region}&"
           f"paymentMethodId=%2Fpayments%2F{payment_method}&"
           f"cryptoCurrencyId={crypto_id}&"
           f"fiatCurrencyId={fiat_id}&"
           f"amount={amount}&"
           f"frequency=undefined&"
           f"{providers_str}&"
           f"walletAddress=0x8f16c9880fb929d072441d7a310185fa9d7269c5&"
           f"sdk=1.28.3&context=browser&keys=")
    
    headers = {
        "Content-Type": "application/json",
        "Accept": "application/json",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
        "Authorization": "Bearer YOUR_API_KEY"
    }
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        data = response.json()
        return data
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except Exception as err:
        print(f"Other error occurred: {err}")
    return None

def process_data(data, crypto_id_mapping, current_crypto_id):
    data_quote = []

    crypto_name = crypto_id_mapping.get(current_crypto_id, 'Unknown Crypto')

    if 'success' in data:
        for rank, entry in enumerate(data['success'], start=1):
            provider = entry.get('provider')
            quote_data = entry.get('quote', {})
            native_apple_pay = entry.get('nativeApplePay', {})
            
            data_quote.append({
                'Ranking': rank,
                'Provider': provider,
                'Crypto Name': quote_data.get('crypto', {}).get('name', 'N/A'),
                'Crypto Symbol': quote_data.get('crypto', {}).get('symbol', 'N/A'),
                'Currency': quote_data.get('fiat', {}).get('symbol'),
                'Decimals': quote_data.get('fiat', {}).get('decimals'),
                'Amount In': quote_data.get('amountIn'),
                'Amount Out': quote_data.get('amountOut'),
                'Exchange Rate': quote_data.get('exchangeRate'),
                'Network Fee': quote_data.get('networkFee'),
                'Provider Fee': quote_data.get('providerFee'),
                'Extra Fee': quote_data.get('extraFee'),
                'Receiver': quote_data.get('receiver'),
                'Payment Method': quote_data.get('paymentMethod'),
                'Best Rate': quote_data.get('bestRate'),
                'Apple Pay Supported': native_apple_pay.get('supported', 'N/A'),
                'Crypto Name 2': crypto_name
            })

        df_quote = pd.DataFrame(data_quote)
        df_quote['Provider'] = df_quote['Provider'].str.split('/').str.get(-1)
        df_quote['Provider'] = df_quote['Provider'].str.replace(r'-b$', '', regex=True)
        return df_quote
    else:
        print("Failed to fetch quote data.")
        return pd.DataFrame()

# Main script
expanded_data_list = []

for region in regions:
    for payment_method in payment_methods[region]:
        for crypto_name, crypto_id in crypto_ids.items():
            fiat_id = fiat_currencies[region]
            for price in price_points:
                print(f"Fetching data for {crypto_name} in {region} with payment method {payment_method} and amount {price}")
                data = fetch_data(region, payment_method, crypto_id, fiat_id, price)
                if data:
                    df_quote = process_data(data, crypto_name_mapping, crypto_id)
                    if not df_quote.empty:
                        df_quote['region'] = region
                        expanded_data_list.append(df_quote)
                        print(f"Data fetched successfully for amount {price}")
                    else:
                        print("No valid quote data found.")
                else:
                    print("Failed to fetch data.")
                
                time.sleep(1)

df_expanded = pd.concat(expanded_data_list, ignore_index=True)


In [ ]:
df_expanded.head()

In [ ]:
project_id='hypnotic-matter-376610'
dataset_id = 'Moonpay'  
table = 'scraped_quote_data'

# Convert numeric columns to the appropriate types
df_expanded['Amount In'] = pd.to_numeric(df_expanded['Amount In'], errors='coerce')
df_expanded['Amount Out'] = pd.to_numeric(df_expanded['Amount Out'], errors='coerce')
df_expanded['Exchange Rate'] = pd.to_numeric(df_expanded['Exchange Rate'], errors='coerce')
df_expanded['Network Fee'] = pd.to_numeric(df_expanded['Network Fee'], errors='coerce')
df_expanded['Provider Fee'] = pd.to_numeric(df_expanded['Provider Fee'], errors='coerce')
df_expanded['Extra Fee'] = pd.to_numeric(df_expanded['Extra Fee'], errors='coerce')

# Convert to string
df_expanded['region'] = df_expanded['region'].astype('str')
df_expanded['Crypto Name 2'] = df_expanded['Crypto Name 2'].astype('str')

# Optional: Convert categorical columns if needed
df_expanded['Currency'] = df_expanded['Currency'].astype('category')
df_expanded['Provider'] = df_expanded['Provider'].astype('category')
df_expanded['Payment Method'] = df_expanded['Payment Method'].astype('category')

df_expanded['Exchange Rate'] = df_expanded['Exchange Rate'].apply(pd.to_numeric, errors='coerce')

# Cleaning and adding timestamp
df_expanded['timestamp'] = datetime.utcnow()
df_expanded['Payment Method'] = df_expanded['Payment Method'].replace('credit_debit_card', 'dedit_crebit_card')

df_expanded = df_expanded.drop(columns=['Receiver', 'Best Rate', 'Decimals','Apple Pay Supported','Crypto Name'])
df_expanded = df_expanded.rename(columns={'Crypto Name 2': 'Crypto Name'})

# Save to BigQuery
try:
    to_gbq(df_expanded, f"{dataset_id}.{table}", project_id=project_id, if_exists='append')
    print("Data successfully saved to BigQuery.")
except Exception as e:
    print(f"An error occurred: {e}")